# Prompting with Prompt Templates for LLM Input / Output with LangChain

## Install OpenAI, HuggingFace and LangChain dependencies

In [ ]:
!pip install langchain==0.3.11
!pip install langchain-openai==0.2.12
!pip install langchain-community==0.3.11

## Enter API Tokens

#### Enter your Open AI Key here

You can get the key from [here](https://platform.openai.com/api-keys) after creating an account or signing in

In [ ]:
from getpass import getpass

OPENAI_KEY = getpass('Enter Open AI API Key: ')

In [5]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY

## Chat Models and LLMs

Large Language Models (LLMs) are a core component of LangChain. LangChain does not implement or build its own LLMs. It provides a standard API for interacting with almost every LLM out there.

There are lots of LLM providers (OpenAI, Hugging Face, etc) - the LLM class is designed to provide a standard interface for all of them.

## Accessing Commercial LLMs like ChatGPT

In [6]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

## Prompt Templates
Prompt templates are pre-designed formats used to generate prompts for language models. These templates can include instructions, few-shot examples, and specific contexts and questions suited for particular tasks.

LangChain provides tools for creating and using prompt templates. It aims to develop model-agnostic templates to facilitate the reuse of existing templates across different language models. Typically, these models expect prompts in the form of either a string or a list of chat messages.

### Types of Prompt Templates

- **PromptTemplate:**
  - Used for creating string-based prompts.
  - Utilizes Python's `str.format` syntax for templating, supporting any number of variables, including scenarios with no variables.

- **ChatPromptTemplate:**
  - Designed for chat models, where the prompt consists of a list of chat messages.
  - Each chat message includes content and a role parameter. For instance, in the OpenAI Chat Completions API, a chat message could be assigned to an AI assistant, a human, or a system role.
- **FewShotChatMessagePromptTemplate**
  - A few-shot prompt template can be constructed from a set of examples


### PromptTemplate

We can use `PromptTemplate` to create a template for a string prompt.

By default, `PromptTemplate` uses Python's `str.format` syntax for templating.

You can create custom prompt templates that format the prompt in any way you want. For more information, see [Prompt Template Composition](https://python.langchain.com/v0.1/docs/modules/model_io/prompts/composition/).

In [7]:
from langchain.prompts import PromptTemplate

# Simple prompt

prompt = """Explain to me what is Generative AI in 3 bullet points?"""
prompt_template = PromptTemplate.from_template(prompt)
prompt_template

PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Explain to me what is Generative AI in 3 bullet points?')

In [8]:
prompt_template.format()

'Explain to me what is Generative AI in 3 bullet points?'

In [9]:
response = chatgpt.invoke(prompt_template.format())
print(response.content)

- **Definition**: Generative AI refers to a class of artificial intelligence models that can create new content, such as text, images, music, or videos, by learning patterns and structures from existing data.

- **Techniques**: It often employs advanced techniques like deep learning, particularly neural networks such as Generative Adversarial Networks (GANs) and Transformer models, to generate high-quality and coherent outputs.

- **Applications**: Generative AI is used in various fields, including creative arts (e.g., art and music generation), content creation (e.g., writing and summarization), and even in practical applications like drug discovery and game design.


In [10]:
# more complex prompt with placeholders
prompt = """Explain to me briefly about {topic} in {language}."""

prompt_template = PromptTemplate.from_template(prompt)
prompt_template

PromptTemplate(input_variables=['language', 'topic'], input_types={}, partial_variables={}, template='Explain to me briefly about {topic} in {language}.')

In [11]:
inputs = [("Generative AI", "english"),
          ("Artificial Intelligence", "hindi"),
          ("Deep Learning", "german")]

prompts = [prompt_template.format(topic=topic, language=language) for topic, language in inputs]
prompts

['Explain to me briefly about Generative AI in english.',
 'Explain to me briefly about Artificial Intelligence in hindi.',
 'Explain to me briefly about Deep Learning in german.']

In [12]:
# use map to run on multiple prompts in one go
responses = chatgpt.map().invoke(prompts)

In [13]:
responses

[AIMessage(content='Generative AI refers to a class of artificial intelligence models that can create new content, such as text, images, music, or even videos, based on the patterns and information they have learned from existing data. These models, like GPT-3 for text or DALL-E for images, use deep learning techniques to understand and generate outputs that resemble human-created content. Generative AI has applications in various fields, including art, entertainment, marketing, and more, enabling innovative solutions and creative possibilities.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 99, 'prompt_tokens': 18, 'total_tokens': 117, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'stop

In [14]:
for response in responses:
  print(response.content)
  print('-----')

Generative AI refers to a class of artificial intelligence models that can create new content, such as text, images, music, or even videos, based on the patterns and information they have learned from existing data. These models, like GPT-3 for text or DALL-E for images, use deep learning techniques to understand and generate outputs that resemble human-created content. Generative AI has applications in various fields, including art, entertainment, marketing, and more, enabling innovative solutions and creative possibilities.
-----
कृत्रिम बुद्धिमत्ता (Artificial Intelligence या AI) एक ऐसी तकनीक है जो मशीनों और कंप्यूटरों को मानव जैसी बुद्धिमत्ता और सोचने की क्षमता प्रदान करती है। इसका उद्देश्य मशीनों को इस तरह से विकसित करना है कि वे डेटा को समझ सकें, सीख सकें, निर्णय ले सकें और समस्याओं को हल कर सकें। 

AI का उपयोग विभिन्न क्षेत्रों में किया जाता है, जैसे कि स्वास्थ्य सेवा, वित्त, परिवहन, और ग्राहक सेवा। उदाहरण के लिए, चैटबॉट्स, स्वायत्त वाहन, और व्यक्तिगत अनुशंसा प्रणाली सभी AI के उ

### ChatPromptTemplate

The standard prompt format to [chat models](https://python.langchain.com/v0.1/docs/modules/model_io/chat/) is a list of [chat messages](https://python.langchain.com/v0.1/docs/modules/model_io/chat/message_types/).

Each chat message is associated with content, and an additional parameter called `role`. For example, in the OpenAI Chat Completions API, a chat message can be associated with an AI assistant, a human or a system role.

In [15]:
from langchain_core.prompts import ChatPromptTemplate

# simple prompt with placeholders
prompt = """Explain to me briefly about {topic}."""

chat_template = ChatPromptTemplate.from_template(prompt)
chat_template

ChatPromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Explain to me briefly about {topic}.'), additional_kwargs={})])

In [16]:
topics = ['mortgage', 'fractional real estate', 'commercial real estate']
prompts = [chat_template.format(topic=topic) for topic in topics]
prompts

['Human: Explain to me briefly about mortgage.',
 'Human: Explain to me briefly about fractional real estate.',
 'Human: Explain to me briefly about commercial real estate.']

In [17]:
responses = chatgpt.map().invoke(prompts)
for response in responses:
  print(response.content)
  print('-----')

A mortgage is a type of loan specifically used to purchase real estate. In a mortgage agreement, the borrower receives funds from a lender to buy a home and agrees to repay the loan over a specified period, typically 15 to 30 years. The property itself serves as collateral, meaning if the borrower fails to make payments, the lender can take possession of the property through a process called foreclosure. Mortgages usually involve monthly payments that cover both principal (the amount borrowed) and interest (the cost of borrowing), and they may also include property taxes and insurance.
-----
Fractional real estate refers to a shared ownership model where multiple investors collectively own a property, allowing them to share the costs and benefits associated with it. Each owner holds a fraction of the property, which entitles them to a proportional share of the income generated (like rental income) and appreciation in value. This model makes real estate investment more accessible, as in

In [18]:
responses[0]

AIMessage(content='A mortgage is a type of loan specifically used to purchase real estate. In a mortgage agreement, the borrower receives funds from a lender to buy a home and agrees to repay the loan over a specified period, typically 15 to 30 years. The property itself serves as collateral, meaning if the borrower fails to make payments, the lender can take possession of the property through a process called foreclosure. Mortgages usually involve monthly payments that cover both principal (the amount borrowed) and interest (the cost of borrowing), and they may also include property taxes and insurance.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 112, 'prompt_tokens': 16, 'total_tokens': 128, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 

In [19]:
# more complex prompt with a series of messages
messages = [
        ("system", "Act as an expert in real estate and provide brief answers"),
        ("human", "what is your name?"),
        ("ai", "my name is AIBot"),
        ("human", "{user_prompt}"),
]
chat_template = ChatPromptTemplate.from_messages(messages)
chat_template

ChatPromptTemplate(input_variables=['user_prompt'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Act as an expert in real estate and provide brief answers'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='what is your name?'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='my name is AIBot'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_prompt'], input_types={}, partial_variables={}, template='{user_prompt}'), additional_kwargs={})])

In [20]:
text_prompts = ["what is your name?",
                "explain commercial real estate to me"]
chat_prompts = [chat_template.format(user_prompt=prompt) for prompt in text_prompts]
chat_prompts

['System: Act as an expert in real estate and provide brief answers\nHuman: what is your name?\nAI: my name is AIBot\nHuman: what is your name?',
 'System: Act as an expert in real estate and provide brief answers\nHuman: what is your name?\nAI: my name is AIBot\nHuman: explain commercial real estate to me']

In [21]:
print(chat_prompts[0])

System: Act as an expert in real estate and provide brief answers
Human: what is your name?
AI: my name is AIBot
Human: what is your name?


In [22]:
print(chat_prompts[1])

System: Act as an expert in real estate and provide brief answers
Human: what is your name?
AI: my name is AIBot
Human: explain commercial real estate to me


In [23]:
responses = chatgpt.map().invoke(chat_prompts)
for response in responses:
  print(response.content)
  print('-----')

AI: I am AIBot, your real estate expert. How can I assist you today?
-----
Commercial real estate (CRE) refers to properties used for business purposes, including office buildings, retail spaces, warehouses, and industrial properties. Unlike residential real estate, which is primarily for living, CRE is intended to generate profit through rental income or capital appreciation. It often involves longer lease terms and can be more complex in terms of financing and management.
-----


In [24]:
messages = [
        ("system", "Act as an expert in real estate and provide very detailed answers with examples"),
        ("human", "what is your name?"),
        ("ai", "my name is AIBot"),
        ("human", "{user_prompt}"),
]
chat_template = ChatPromptTemplate.from_messages(messages)
text_prompts = ["what is your name?", "explain commercial real estate to me"]
chat_prompts = [chat_template.format(user_prompt=prompt) for prompt in text_prompts]
chat_prompts

['System: Act as an expert in real estate and provide very detailed answers with examples\nHuman: what is your name?\nAI: my name is AIBot\nHuman: what is your name?',
 'System: Act as an expert in real estate and provide very detailed answers with examples\nHuman: what is your name?\nAI: my name is AIBot\nHuman: explain commercial real estate to me']

In [25]:
responses = chatgpt.map().invoke(chat_prompts)
for response in responses:
  print(response.content)
  print('-----')

AI: My name is AIBot, and I'm here to assist you with any questions you have about real estate. Whether you're looking for information on buying, selling, investing, or understanding market trends, feel free to ask!
-----
Commercial real estate (CRE) refers to properties that are used exclusively for business purposes. This category of real estate is distinct from residential real estate, which is primarily used for living purposes. Understanding commercial real estate involves recognizing its various types, investment potential, market dynamics, and the factors that influence its value.

### Types of Commercial Real Estate

1. **Office Buildings**: These properties are designed for business operations and can range from single-tenant buildings to large skyscrapers. They are often categorized into:
   - **Class A**: High-quality buildings in prime locations with top amenities (e.g., high-rise buildings in downtown areas).
   - **Class B**: Older buildings that are still functional but 

#### PromptTemplate and ChatPromptTemplate supports LCEL

`PromptTemplate` and `ChatPromptTemplate` implement the [Runnable interface](https://python.langchain.com/v0.1/docs/expression_language/interface/), the basic building block of the LangChain Expression Language (LCEL). This means they support `invoke`, `ainvoke`, `stream`, `astream`, `batch`, `abatch`, `astream_log` calls.

`PromptTemplate` accepts a dictionary (of the prompt variables) and returns a `StringPromptValue`. A `ChatPromptTemplate` accepts a dictionary and returns a `ChatPromptValue`.

In [26]:
text_prompts = ["what is your name?", "explain commercial real estate to me"]
chat_prompts = [chat_template.invoke({'user_prompt' : prompt}) for prompt in text_prompts]
chat_prompts

[ChatPromptValue(messages=[SystemMessage(content='Act as an expert in real estate and provide very detailed answers with examples', additional_kwargs={}, response_metadata={}), HumanMessage(content='what is your name?', additional_kwargs={}, response_metadata={}), AIMessage(content='my name is AIBot', additional_kwargs={}, response_metadata={}), HumanMessage(content='what is your name?', additional_kwargs={}, response_metadata={})]),
 ChatPromptValue(messages=[SystemMessage(content='Act as an expert in real estate and provide very detailed answers with examples', additional_kwargs={}, response_metadata={}), HumanMessage(content='what is your name?', additional_kwargs={}, response_metadata={}), AIMessage(content='my name is AIBot', additional_kwargs={}, response_metadata={}), HumanMessage(content='explain commercial real estate to me', additional_kwargs={}, response_metadata={})])]

In [27]:
chat_prompts[1]

ChatPromptValue(messages=[SystemMessage(content='Act as an expert in real estate and provide very detailed answers with examples', additional_kwargs={}, response_metadata={}), HumanMessage(content='what is your name?', additional_kwargs={}, response_metadata={}), AIMessage(content='my name is AIBot', additional_kwargs={}, response_metadata={}), HumanMessage(content='explain commercial real estate to me', additional_kwargs={}, response_metadata={})])

In [28]:
print(chat_prompts[1].to_string())

System: Act as an expert in real estate and provide very detailed answers with examples
Human: what is your name?
AI: my name is AIBot
Human: explain commercial real estate to me


In [29]:
chat_prompts[1].to_messages()

[SystemMessage(content='Act as an expert in real estate and provide very detailed answers with examples', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is your name?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='my name is AIBot', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='explain commercial real estate to me', additional_kwargs={}, response_metadata={})]

In [30]:
responses = chatgpt.map().invoke(chat_prompts)
for response in responses:
  print(response.content)
  print('-----')

I am known as AIBot. How can I assist you with your real estate inquiries today?
-----
Commercial real estate (CRE) refers to properties that are used exclusively for business purposes or to provide a workspace rather than as a residence. This sector of real estate encompasses a wide range of property types, each serving different functions and catering to various industries. Here’s a detailed breakdown of commercial real estate, including its types, investment considerations, and examples.

### Types of Commercial Real Estate

1. **Office Buildings**: 
   - **Description**: These properties are designed for business operations and can range from single-tenant buildings to multi-tenant skyscrapers.
   - **Examples**: Corporate headquarters, co-working spaces, and medical office buildings.
   - **Investment Considerations**: Location, tenant mix, lease terms, and building amenities are crucial factors. For instance, a well-located office building in a metropolitan area may attract highe

### FewShotChatMessagePromptTemplate

This is a Chat prompt template that supports few-shot examples.

The high level structure of produced by this prompt template is a list of messages consisting of prefix message(s), example message(s), and suffix message(s).

This structure enables creating a conversation with intermediate examples like:

```
System: You are a helpful AI Assistant
Human: What is 2+2?
AI: 4
Human: What is 2+3?
AI: 5
Human: What is 4+4?
```

Define the few-shot examples you'd like to include.

In [31]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

example_data = [
    ("""Wimbledon 2023 Women’s Final.
    Marketa Vondrousova beats Ons Jabeur to win her maiden Grand Slam title""",
     'Sports'),
    ("""It's time to call it a day as India had another medal-laden day at the Asian Athletics Championships
    in Bangkok. Murali Sreeshankar's Olympic qualifying jump grabbed the limelight while 400m mixed relay
    team added's India's sixth gold medal! There will be plenty of medals up for grab on the final day of
    the tournament as well with Jyothi Yarraji, women's and men's 400m relay teams in action""",
     'Sports'),
    ("""Kicking things off with the Galaxy Tab S9 FE Plus. It's going to be powered by the Exynos 1380
    with 8GB of RAM, and pack a 12.4-inch display. The slate will measure 285.4 x 185.4 x 6.54 mm and
    feature dual speakers, a side-mounted fingerprint sensor. It will come in Grey, Light Green,
    Light Pink, and Silver color options.""",
     'Technology'),
    ("""The share of the cigarette business to ITC’s business in revenue terms has dropped from 47
    per cent in FY13 to 37 per cent in FY23 on a larger revenue base.
    According to the company’s most recent annual report, the leadership position has been sustained
    "through its unwavering focus on nurturing a future-ready portfolio of world-class products anchored on
    its integrated seed to smoke value chain, superior consumer insights, robust innovation pipeline and
    world-class product development capabilities." """,
     'Business')
]

example_data

[('Wimbledon 2023 Women’s Final.\n    Marketa Vondrousova beats Ons Jabeur to win her maiden Grand Slam title',
  'Sports'),
 ("It's time to call it a day as India had another medal-laden day at the Asian Athletics Championships\n    in Bangkok. Murali Sreeshankar's Olympic qualifying jump grabbed the limelight while 400m mixed relay\n    team added's India's sixth gold medal! There will be plenty of medals up for grab on the final day of\n    the tournament as well with Jyothi Yarraji, women's and men's 400m relay teams in action",
  'Sports'),
 ("Kicking things off with the Galaxy Tab S9 FE Plus. It's going to be powered by the Exynos 1380\n    with 8GB of RAM, and pack a 12.4-inch display. The slate will measure 285.4 x 185.4 x 6.54 mm and\n    feature dual speakers, a side-mounted fingerprint sensor. It will come in Grey, Light Green,\n    Light Pink, and Silver color options.",
  'Technology'),
 ('The share of the cigarette business to ITC’s business in revenue terms has dropped f

In [32]:
example_data_formatted = [{"input": input, "output": output}
                            for input, output in example_data]
example_data_formatted

[{'input': 'Wimbledon 2023 Women’s Final.\n    Marketa Vondrousova beats Ons Jabeur to win her maiden Grand Slam title',
  'output': 'Sports'},
 {'input': "It's time to call it a day as India had another medal-laden day at the Asian Athletics Championships\n    in Bangkok. Murali Sreeshankar's Olympic qualifying jump grabbed the limelight while 400m mixed relay\n    team added's India's sixth gold medal! There will be plenty of medals up for grab on the final day of\n    the tournament as well with Jyothi Yarraji, women's and men's 400m relay teams in action",
  'output': 'Sports'},
 {'input': "Kicking things off with the Galaxy Tab S9 FE Plus. It's going to be powered by the Exynos 1380\n    with 8GB of RAM, and pack a 12.4-inch display. The slate will measure 285.4 x 185.4 x 6.54 mm and\n    feature dual speakers, a side-mounted fingerprint sensor. It will come in Grey, Light Green,\n    Light Pink, and Silver color options.",
  'output': 'Technology'},
 {'input': 'The share of the c

Assemble them into the few-shot prompt template.

In [33]:
data_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=data_prompt,
    examples=example_data_formatted,
)

few_shot_prompt

FewShotChatMessagePromptTemplate(examples=[{'input': 'Wimbledon 2023 Women’s Final.\n    Marketa Vondrousova beats Ons Jabeur to win her maiden Grand Slam title', 'output': 'Sports'}, {'input': "It's time to call it a day as India had another medal-laden day at the Asian Athletics Championships\n    in Bangkok. Murali Sreeshankar's Olympic qualifying jump grabbed the limelight while 400m mixed relay\n    team added's India's sixth gold medal! There will be plenty of medals up for grab on the final day of\n    the tournament as well with Jyothi Yarraji, women's and men's 400m relay teams in action", 'output': 'Sports'}, {'input': "Kicking things off with the Galaxy Tab S9 FE Plus. It's going to be powered by the Exynos 1380\n    with 8GB of RAM, and pack a 12.4-inch display. The slate will measure 285.4 x 185.4 x 6.54 mm and\n    feature dual speakers, a side-mounted fingerprint sensor. It will come in Grey, Light Green,\n    Light Pink, and Silver color options.", 'output': 'Technology

In [34]:
print(few_shot_prompt.format())

Human: Wimbledon 2023 Women’s Final.
    Marketa Vondrousova beats Ons Jabeur to win her maiden Grand Slam title
AI: Sports
Human: It's time to call it a day as India had another medal-laden day at the Asian Athletics Championships
    in Bangkok. Murali Sreeshankar's Olympic qualifying jump grabbed the limelight while 400m mixed relay
    team added's India's sixth gold medal! There will be plenty of medals up for grab on the final day of
    the tournament as well with Jyothi Yarraji, women's and men's 400m relay teams in action
AI: Sports
Human: Kicking things off with the Galaxy Tab S9 FE Plus. It's going to be powered by the Exynos 1380
    with 8GB of RAM, and pack a 12.4-inch display. The slate will measure 285.4 x 185.4 x 6.54 mm and
    feature dual speakers, a side-mounted fingerprint sensor. It will come in Grey, Light Green,
    Light Pink, and Silver color options.
AI: Technology
Human: The share of the cigarette business to ITC’s business in revenue terms has dropped from

Finally, assemble your final prompt and use it with a LLM.

In [35]:
final_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Classify every new news article using a similar format as shown below"),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)
final_prompt_template

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Classify every new news article using a similar format as shown below'), additional_kwargs={}), FewShotChatMessagePromptTemplate(examples=[{'input': 'Wimbledon 2023 Women’s Final.\n    Marketa Vondrousova beats Ons Jabeur to win her maiden Grand Slam title', 'output': 'Sports'}, {'input': "It's time to call it a day as India had another medal-laden day at the Asian Athletics Championships\n    in Bangkok. Murali Sreeshankar's Olympic qualifying jump grabbed the limelight while 400m mixed relay\n    team added's India's sixth gold medal! There will be plenty of medals up for grab on the final day of\n    the tournament as well with Jyothi Yarraji, women's and men's 400m relay teams in action", 'output': 'Sports'}, {'input': "Kicking things off with the Galaxy Tab S9 FE Plus. It'

In [36]:
docs = ["""Arsenal finally completed the signing of Rice as they announced the player's arrival on Saturday.
Right after completing his move, Rice revealed that he is a great admirer of Arteta
and that the Spaniard's presence in the Arsenal dressing room is one of the main reasons
why he decided to join the North London club.""",
        """A heat wave sweeping across Europe is prompting health warnings as millions
        of people endure sweltering temperatures. Italy has issued red alerts in 16 cities with
        the country's weather centre warning of record heat especially in southern regions.
        Parts of Greece are forecast to hit highs of 44 degrees celsius, while France,
        Germany, Spain and Poland are also enduring extreme temperatures.
        Scientists say greenhouse gases and the El Niño weather pattern are raising global temperatures.
        """]

final_prompts = [final_prompt_template.format(input=doc) for doc in docs]
responses = chatgpt.map().invoke(final_prompts)

In [37]:
responses[0]

AIMessage(content='AI: Sports', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 431, 'total_tokens': 434, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'stop', 'logprobs': None}, id='run-2bd501fc-6fcc-42a5-81eb-fa82f40be81f-0', usage_metadata={'input_tokens': 431, 'output_tokens': 3, 'total_tokens': 434, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [38]:
for response in responses:
  print(response.content)
  print('-----')

AI: Sports
-----
AI: Environment
-----


### Partial prompt templates

Like other methods, it can make sense to "partial" a prompt template - e.g. pass in a subset of the required data values, as to create a new prompt template which expects only the remaining subset of data values.

Imagine you get some inputs initially and then some other inputs for the prompts later, you can use the `partial`function to add some of the input data first and the rest of the input data later

In [39]:
from datetime import datetime

def _get_datetime():
    now = datetime.now()
    return now.strftime("%m/%d/%Y")

In [40]:
prompt_txt = """Tell me a joke about {topic} on the day {date}"""
prompt = ChatPromptTemplate.from_template(prompt_txt)

In [41]:
prompt = prompt.partial(date=_get_datetime)
prompt

ChatPromptTemplate(input_variables=['topic'], input_types={}, partial_variables={'date': <function _get_datetime at 0x7a896a606200>}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['date', 'topic'], input_types={}, partial_variables={}, template='Tell me a joke about {topic} on the day {date}'), additional_kwargs={})])

In [42]:
topics = ['Engineers', 'Statisticians']
final_prompts = [prompt.format(topic=topic) for topic in topics]
final_prompts

['Human: Tell me a joke about Engineers on the day 11/11/2024',
 'Human: Tell me a joke about Statisticians on the day 11/11/2024']

In [43]:
responses = chatgpt.map().invoke(final_prompts)
for response in responses:
  print(response.content)
  print('-----')

Why did the engineer bring a ladder to work on 11/11/2024?

Because they heard the job was on another level!
-----
Why did the statistician bring a ladder to the bar on 11/11/2024?

Because they heard the drinks were on the house, and they wanted to calculate the "mean" height!
-----
